In [1]:
import json
import sys
import re
import os
from collections import defaultdict


def remove_unlabel(annotation):
    data = sorted(annotation['labels'], key=lambda x: x['pos'][0])
    flag = 0
    unlabel = []
    for item in data:
        # if re.search('鉴别诊断',annotation['content'][item['pos'][0]:item['pos'][1] + 1]):
        #     print('*******',annotation['id'])

        if re.match(r'^鉴别诊断$', annotation['content'][item['pos'][0]:item['pos'][1] + 1]) or item['category'] == 44:
            # print('---------------', annotation['id'])
            # print(annotation['content'][item['pos'][0]:item['pos'][1] + 1])

            flag = 1
            unlabel.append(item['id'])
        elif flag == 1:
            # print(annotation['content'][item['pos'][0]:item['pos'][1] + 1], annotation['id'])
            if item['category'] == 28 or item['category'] == 44:
                flag = 0
            unlabel.append(item['id'])
        elif re.match(r'^\s+$', annotation['content'][item['pos'][0]:item['pos'][1] + 1]):
            unlabel.append(item['id'])

    rel = [i for i in annotation['relations'] if (i['src'] not in unlabel and i['dst'] not in unlabel)]
    lab = [i for i in annotation['labels'] if i['id'] not in unlabel]
    annotation['labels'] = lab
    annotation['relations'] = rel

    return annotation

def train2anno(train):
    result = {'concepts': []}
    train = remove_unlabel(train)
    sorted_label = sorted(train['labels'], key=lambda x: x['pos'][0])
    temp = {'id': 0, 'pos': [0, 0], 'relation': [], 'timestamps': 0}
    for j, item in enumerate(sorted_label):
        result['concepts'].append(
            {'id': item['id'], 'pos': [item['pos'][0], item['pos'][1] + 1], 'relations': [], 'timestamps': item['id'],
             'category':item['category'],'text':train['content'][item['pos'][0]:item['pos'][1] + 1]})

    for j, item in enumerate(train['relations']):
        src = item['src']
        dst = item['dst']
        for ii in result['concepts']:
            if ii['timestamps'] == dst:
                for jj in result['concepts']:
                    if jj['timestamps'] == src:
                        ii['relations'].append({"text": jj['text'], "src_id": jj['id'], 'type': item['category']})

    anno = {'NLPResult': result, 'rawText': train['content']}

    return anno

In [2]:
# 使用列表生成式， 获得10个json中的序号，然后获得每篇json中的查找字典

In [3]:
# c = [i for i in range(2, 5)]
# c
# for i in c:
#     print(i)

In [4]:
# 可以用正则，获得数字, range(a, b+1)， 注意b需要加1
anno_id_list = [i for i in range(37529, 37539)]
anno_id_list

[37529, 37530, 37531, 37532, 37533, 37534, 37535, 37536, 37537, 37538]

### json文件的字符串有问题 -> .json(aldsfjlajf) 后面存在一段乱码

In [5]:
# 正则获取字符串中的数字， a, b,  用于构造list



In [6]:
# 子函数open_json
def open_json(files):
    if files.endswith('.json'):
        with open(files, 'r', encoding='utf8') as f:
            return json.load(f)

    else:
        result = []
        PATH = os.listdir(files)
        for file in PATH:
            with open(files + file, 'r', encoding='utf8') as f:
                result.extend(json.load(f))
        return result


In [7]:
#  主函数
def find_all_checklist_json(json_path):
    """
    输入：一个json地址, 包括10个json的地址
    
    输出：查询节点信息的list，每个json对应不同的查找字典
    [{'id': 106, 'pos': [1, 18], 'relations': [], 'timestamps': 106, 'category': 28, 'text': 'ClinicalDiagnosis'}, ...]
    """
    
    # 未来这句需要用一个函数代替
    anno_id_list = [i for i in range(37529, 37539)]
    
    all_checklist_json_list = [] # 收集10个
    
    for anno_id in anno_id_list:
        data = open_json(json_path)
        a = set()
        for item in data:
            if  item["id"] == anno_id:
                each_check_dict = train2anno(item)['NLPResult']['concepts']
        all_checklist_json_list.append(each_check_dict)
    return all_checklist_json_list
               

In [8]:
json_path = "/Users/synyi/Desktop/test_json_for_checklist/annotation37529-37538.json"
all_checklist_json_list = find_all_checklist_json(json_path)

In [9]:
all_checklist_json_list

[[{'id': 106,
   'pos': [1, 18],
   'relations': [],
   'timestamps': 106,
   'category': 28,
   'text': 'ClinicalDiagnosis'},
  {'id': 1,
   'pos': [20, 22],
   'relations': [],
   'timestamps': 1,
   'category': 7,
   'text': '肝脏'},
  {'id': 108,
   'pos': [22, 25],
   'relations': [],
   'timestamps': 108,
   'category': 11,
   'text': '占位性'},
  {'id': 109,
   'pos': [25, 27],
   'relations': [{'text': '肝脏', 'src_id': 1, 'type': 135},
    {'text': '占位性', 'src_id': 108, 'type': 134}],
   'timestamps': 109,
   'category': 53,
   'text': '病变'},
  {'id': 107,
   'pos': [29, 41],
   'relations': [],
   'timestamps': 107,
   'category': 28,
   'text': 'ImageFinding'},
  {'id': 4,
   'pos': [43, 45],
   'relations': [],
   'timestamps': 4,
   'category': 7,
   'text': '右肺'},
  {'id': 5,
   'pos': [45, 47],
   'relations': [{'text': '右肺', 'src_id': 4, 'type': 135}],
   'timestamps': 5,
   'category': 56,
   'text': '下叶'},
  {'id': 6,
   'pos': [48, 50],
   'relations': [],
   'timestamps': 

In [10]:
len(all_checklist_json_list)

10

In [11]:
all_checklist_json_list[0]

[{'id': 106,
  'pos': [1, 18],
  'relations': [],
  'timestamps': 106,
  'category': 28,
  'text': 'ClinicalDiagnosis'},
 {'id': 1,
  'pos': [20, 22],
  'relations': [],
  'timestamps': 1,
  'category': 7,
  'text': '肝脏'},
 {'id': 108,
  'pos': [22, 25],
  'relations': [],
  'timestamps': 108,
  'category': 11,
  'text': '占位性'},
 {'id': 109,
  'pos': [25, 27],
  'relations': [{'text': '肝脏', 'src_id': 1, 'type': 135},
   {'text': '占位性', 'src_id': 108, 'type': 134}],
  'timestamps': 109,
  'category': 53,
  'text': '病变'},
 {'id': 107,
  'pos': [29, 41],
  'relations': [],
  'timestamps': 107,
  'category': 28,
  'text': 'ImageFinding'},
 {'id': 4,
  'pos': [43, 45],
  'relations': [],
  'timestamps': 4,
  'category': 7,
  'text': '右肺'},
 {'id': 5,
  'pos': [45, 47],
  'relations': [{'text': '右肺', 'src_id': 4, 'type': 135}],
  'timestamps': 5,
  'category': 56,
  'text': '下叶'},
 {'id': 6,
  'pos': [48, 50],
  'relations': [],
  'timestamps': 6,
  'category': 11,
  'text': '多发'},
 {'id': 1